In [1]:
import duckdb
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
con = duckdb.connect("data/ust.db")
con.execute("SHOW TABLES").df()

,name
0,auctioned_securities


In [3]:
df = con.execute("""
    select *
    from auctioned_securities
""").df()

In [4]:
# df.columns.tolist()
d = df[[
    'cusip','issuedate','maturitydate','interestrate',
    'auctiondate','announcementdate','dateddate',
    'term','tips','type'
]]

In [5]:
d.tail(10)

,cusip,issuedate,maturitydate,interestrate,auctiondate,announcementdate,dateddate,term,tips,type
144,91282CNU1,2025-08-15T00:00:00,2028-08-15T00:00:00,3.625000,2025-08-05T00:00:00,2025-07-30T00:00:00,2025-08-15T00:00:00,3-Year,No,Note
145,91282CNV9,2025-09-02T00:00:00,2027-08-31T00:00:00,3.625000,2025-08-26T00:00:00,2025-08-21T00:00:00,2025-08-31T00:00:00,2-Year,No,Note
146,91282CNW7,2025-09-02T00:00:00,2032-08-31T00:00:00,3.875000,2025-08-28T00:00:00,2025-08-21T00:00:00,2025-08-31T00:00:00,7-Year,No,Note
147,91282CNX5,2025-09-02T00:00:00,2030-08-31T00:00:00,3.625000,2025-08-27T00:00:00,2025-08-21T00:00:00,2025-08-31T00:00:00,5-Year,No,Note
148,91282CNY3,2025-09-15T00:00:00,2028-09-15T00:00:00,3.375000,2025-09-09T00:00:00,2025-09-04T00:00:00,2025-09-15T00:00:00,3-Year,No,Note
149,91282CNZ0,2025-09-30T00:00:00,2032-09-30T00:00:00,3.875000,2025-09-25T00:00:00,2025-09-18T00:00:00,2025-09-30T00:00:00,7-Year,No,Note
150,91282CPA3,2025-09-30T00:00:00,2030-09-30T00:00:00,3.625000,2025-09-24T00:00:00,2025-09-18T00:00:00,2025-09-30T00:00:00,5-Year,No,Note
151,91282CPB1,2025-09-30T00:00:00,2027-09-30T00:00:00,3.500000,2025-09-23T00:00:00,2025-09-18T00:00:00,2025-09-30T00:00:00,2-Year,No,Note
152,91282CPC9,2025-10-15T00:00:00,2028-10-15T00:00:00,3.500000,2025-10-07T00:00:00,2025-10-02T00:00:00,2025-10-15T00:00:00,3-Year,No,Note
153,91282CPH8,2025-10-31T00:00:00,2030-10-15T00:00:00,1.125000,2025-10-23T00:00:00,2025-10-16T00:00:00,2025-10-15T00:00:00,5-Year,Yes,TIPS
